In [ ]:
import keras

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('imdb_10000.csv')

In [ ]:
data

In [ ]:
x = data.review
y = data.sentiment

In [ ]:
len(max(x))

In [ ]:
y

In [ ]:
#positive = 1 negative = 0
for i in range(0,len(y)):
    if y[i] =='positive':
        y[i] = 1
    else:
        y[i] = 0

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

In [ ]:
word_to_index = tokenizer.word_index

In [ ]:
vocab_size = len(word_to_index)+1

In [ ]:
x = tokenizer.texts_to_sequences(x)

In [ ]:
from keras.preprocessing import sequence
x = sequence.pad_sequences(x, maxlen = 1961)

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras import layers


model = Sequential()
model.add(layers.Embedding(vocab_size, 128, input_length=1961))

#model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.Conv1D(128, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))

model.add(layers.Conv1D(64, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())


model.add(layers.Dense(1))

model.summary()

model.compile(optimizer= 'adam',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
y_test

In [ ]:
y_train

In [ ]:
history = model.fit(X_train, y_train,epochs= 15 ,batch_size= 320,validation_split=0.2)

#history = model.fit(X_train, y_train,epochs= 3 ,batch_size= 320,validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(history.history["acc"], label="acc")
plt.plot(history.history["val_acc"], label="val_acc")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")

In [ ]:
import matplotlib.pyplot as plt
#epochs = range(len(history.history['loss']))
plt.figure(figsize=(20,10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.legend(['train', 'val'], loc='upper right')

plt.show()

In [ ]:
test_acc, test_loss = model.evaluate(X_test, y_test)

In [ ]:
test_acc,test_loss

In [ ]:
X_test

In [ ]:
X_train